# 用户人口属性预测
手机用户的人口属性（如性别、年龄、常驻地等）数据一方面可以被用于个性化推荐服务，提升用户体验，另一方面可以用于手机用户群画像分析，帮助厂商了解产品的人群定位，优化产品设计。

In [1]:
###########################
#   -*- coding: utf-8 -*- #
# PHM_data_challenge_2019 #
#     Author: Huet Zhu    #
#       Date:2019.5       #
#    All Rights Reserved  #
###########################

# # 基于机器学习的飞控系统故障诊断方案设计
from __future__ import division

import numpy as np
import xgboost as xgb
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

from tqdm import tqdm

import warnings
import random
warnings.filterwarnings('ignore')

In [2]:
age_test = pd.read_csv("../data/age_test.csv", header = None)
age_train = pd.read_csv("../data/age_train.csv", header = None)
label9521 = pd.read_csv("../trainTestData/label9521.csv", header = None)

# app_info = pd.read_csv("data/app_info.csv", header = None)
# user_app_actived = pd.read_csv("data/user_app_actived.csv", header = None)
# user_app_usage = pd.read_csv("data/user_app_usage.csv")
# user_basic_info = pd.read_csv("data/user_basic_info.csv", header = None)
# user_behavior_info = pd.read_csv("data/user_behavior_info.csv", header = None)

In [3]:
label9521.loc[:, 0].value_counts()

3    469246
4    391322
2    321707
5    241077
6    121580
1     49199
Name: 0, dtype: int64

In [4]:
from scipy import sparse
from scipy.sparse import csr_matrix, hstack, vstack
trainData = sparse.load_npz('../trainTestData/csr_trainData9521.npz')

In [4]:
# import numpy as np
# from sklearn.decomposition import SparsePCA

# pca = SparsePCA(n_components=2000)
# trainData = pca.fit_transform(trainData[:100000].toarray())

# print(pca.get_params)

In [ ]:
k = 0
for k in range(0, 10):
    row_indices = np.random.permutation(trainData.shape[0])
    random.shuffle(row_indices)
    xg_train = xgb.DMatrix(trainData[row_indices[:int(len(row_indices) * 0.8)]],
                           label=label9521.iloc[row_indices[:int(len(row_indices) * 0.8)], 0])
    xg_val = xgb.DMatrix(trainData[row_indices[int(len(row_indices) * 0.8):]], 
                         label=label9521.iloc[row_indices[int(len(row_indices) * 0.8):], 0])
    param = {}
    param['gpu_id'] = 1
    # # param['n_gpus'] = -1
    param['max_bin'] = 512
    param['grow_policy'] = 'depthwise'
    param['tree_method'] = 'gpu_hist'

    param['eta'] = 0.1
    param['min_child_weight'] = 0.8
    param['subsample'] = 0.8

    param['lambda'] = 0.6
#     param['gamma'] = 0.1
    param['max_depth'] = 4
    param['silent'] = 1
    param['nthread'] = 40
    # param['top_k'] = 1500
    # param['scale_pos_weight'] = 9
    # param['missing'] = -999.0

    param['num_class'] = 7
    param['objective'] = 'multi:softprob'
    watchlist = [(xg_train, 'train'), (xg_val, 'val')]
    num_round = 20000

    bst = xgb.train(param, xg_train, num_round, watchlist, early_stopping_rounds=50)
#     bst = xgb.train(param, xg_train, num_round)
    bst.save_model('../model/save_k_flod_9521_'+str(k)+'.model')
    print(k, bst.eval(xg_train), bst.eval(xg_val))
    del bst

[0]	train-merror:0.571135	val-merror:0.570171
Multiple eval metrics have been passed: 'val-merror' will be used for early stopping.

Will train until val-merror hasn't improved in 50 rounds.
[1]	train-merror:0.562198	val-merror:0.561314
[2]	train-merror:0.554218	val-merror:0.553761
[3]	train-merror:0.548864	val-merror:0.548316
[4]	train-merror:0.545863	val-merror:0.544872
[5]	train-merror:0.543298	val-merror:0.542438
[6]	train-merror:0.537692	val-merror:0.536865
[7]	train-merror:0.534241	val-merror:0.533421
[8]	train-merror:0.530353	val-merror:0.529281
[9]	train-merror:0.525937	val-merror:0.525398
[10]	train-merror:0.523837	val-merror:0.522967
[11]	train-merror:0.521287	val-merror:0.520759
[12]	train-merror:0.520695	val-merror:0.519733
[13]	train-merror:0.519275	val-merror:0.518416
[14]	train-merror:0.51712	val-merror:0.516227
[15]	train-merror:0.51516	val-merror:0.514081
[16]	train-merror:0.513188	val-merror:0.512334
[17]	train-merror:0.511896	val-merror:0.510954
[18]	train-merror:0.5

[171]	train-merror:0.429399	val-merror:0.430917
[172]	train-merror:0.429178	val-merror:0.430823
[173]	train-merror:0.428985	val-merror:0.430707
[174]	train-merror:0.428888	val-merror:0.430528
[175]	train-merror:0.428573	val-merror:0.430255
[176]	train-merror:0.428423	val-merror:0.430067
[177]	train-merror:0.428285	val-merror:0.429948
[178]	train-merror:0.428052	val-merror:0.42986
[179]	train-merror:0.427924	val-merror:0.429641
[180]	train-merror:0.427698	val-merror:0.429424
[181]	train-merror:0.427528	val-merror:0.429317
[182]	train-merror:0.427341	val-merror:0.42917
[183]	train-merror:0.427157	val-merror:0.429073
[184]	train-merror:0.426973	val-merror:0.428903
[185]	train-merror:0.426738	val-merror:0.4287
[186]	train-merror:0.42653	val-merror:0.428668
[187]	train-merror:0.426468	val-merror:0.428493
[188]	train-merror:0.426333	val-merror:0.42832
[189]	train-merror:0.426114	val-merror:0.428154
[190]	train-merror:0.425888	val-merror:0.428025
[191]	train-merror:0.425837	val-merror:0.42786

[343]	train-merror:0.407514	val-merror:0.411891
[344]	train-merror:0.407424	val-merror:0.41175
[345]	train-merror:0.407311	val-merror:0.411734
[346]	train-merror:0.407214	val-merror:0.411781
[347]	train-merror:0.407178	val-merror:0.411631
[348]	train-merror:0.407108	val-merror:0.411584
[349]	train-merror:0.407061	val-merror:0.411546
[350]	train-merror:0.407004	val-merror:0.411411
[351]	train-merror:0.406987	val-merror:0.411436
[352]	train-merror:0.406875	val-merror:0.411424
[353]	train-merror:0.406771	val-merror:0.411355
[354]	train-merror:0.406649	val-merror:0.411223
[355]	train-merror:0.406534	val-merror:0.411258
[356]	train-merror:0.406458	val-merror:0.411101
[357]	train-merror:0.406359	val-merror:0.411126
[358]	train-merror:0.406251	val-merror:0.411063
[359]	train-merror:0.406175	val-merror:0.411016
[360]	train-merror:0.406078	val-merror:0.410884
[361]	train-merror:0.406013	val-merror:0.410812
[362]	train-merror:0.405917	val-merror:0.410781
[363]	train-merror:0.405865	val-merror:0.

[515]	train-merror:0.395423	val-merror:0.402011
[516]	train-merror:0.395362	val-merror:0.401939
[517]	train-merror:0.395241	val-merror:0.401829
[518]	train-merror:0.395194	val-merror:0.401832
[519]	train-merror:0.395123	val-merror:0.401776
[520]	train-merror:0.395023	val-merror:0.401792
[521]	train-merror:0.394978	val-merror:0.40181
[522]	train-merror:0.394922	val-merror:0.401657
[523]	train-merror:0.394872	val-merror:0.401607
[524]	train-merror:0.394837	val-merror:0.401622
[525]	train-merror:0.394773	val-merror:0.401559
[526]	train-merror:0.394694	val-merror:0.401465
[527]	train-merror:0.394629	val-merror:0.401418
[528]	train-merror:0.394612	val-merror:0.401374
[529]	train-merror:0.394561	val-merror:0.401403
[530]	train-merror:0.394517	val-merror:0.401403
[531]	train-merror:0.394485	val-merror:0.401343
[532]	train-merror:0.394438	val-merror:0.401318
[533]	train-merror:0.394395	val-merror:0.401309
[534]	train-merror:0.394337	val-merror:0.401312
[535]	train-merror:0.394275	val-merror:0.

[687]	train-merror:0.386948	val-merror:0.395591
[688]	train-merror:0.386932	val-merror:0.395578
[689]	train-merror:0.386919	val-merror:0.395525
[690]	train-merror:0.386878	val-merror:0.395418
[691]	train-merror:0.386819	val-merror:0.395346
[692]	train-merror:0.386777	val-merror:0.395368
[693]	train-merror:0.386736	val-merror:0.395362
[694]	train-merror:0.386691	val-merror:0.395305
[695]	train-merror:0.386643	val-merror:0.395274
[696]	train-merror:0.386576	val-merror:0.39529
[697]	train-merror:0.386545	val-merror:0.395302
[698]	train-merror:0.386542	val-merror:0.395243
[699]	train-merror:0.386465	val-merror:0.39517
[700]	train-merror:0.386432	val-merror:0.395101
[701]	train-merror:0.386392	val-merror:0.395117
[702]	train-merror:0.386337	val-merror:0.395048
[703]	train-merror:0.38631	val-merror:0.395001
[704]	train-merror:0.386256	val-merror:0.394973
[705]	train-merror:0.38621	val-merror:0.394935
[706]	train-merror:0.386162	val-merror:0.394891
[707]	train-merror:0.38614	val-merror:0.3948

...

[396]	train-merror:0.40302	val-merror:0.408535
[397]	train-merror:0.402996	val-merror:0.408585
[398]	train-merror:0.402906	val-merror:0.408472
[399]	train-merror:0.402839	val-merror:0.408334
[400]	train-merror:0.402757	val-merror:0.408312
[401]	train-merror:0.402715	val-merror:0.408381
[402]	train-merror:0.40264	val-merror:0.408275
[403]	train-merror:0.402598	val-merror:0.408253
[404]	train-merror:0.402509	val-merror:0.408181
[405]	train-merror:0.402418	val-merror:0.40809
[406]	train-merror:0.402358	val-merror:0.407961
[407]	train-merror:0.402277	val-merror:0.407889
[408]	train-merror:0.402174	val-merror:0.407757
[409]	train-merror:0.402123	val-merror:0.407785
[410]	train-merror:0.402077	val-merror:0.407644
[411]	train-merror:0.40201	val-merror:0.407541
[412]	train-merror:0.401927	val-merror:0.407469
[413]	train-merror:0.401827	val-merror:0.407371
[414]	train-merror:0.401772	val-merror:0.407224
[415]	train-merror:0.401725	val-merror:0.407149
[416]	train-merror:0.401659	val-merror:0.407

[568]	train-merror:0.392335	val-merror:0.399941
[569]	train-merror:0.392276	val-merror:0.399891
[570]	train-merror:0.392264	val-merror:0.399794
[571]	train-merror:0.39219	val-merror:0.399812
[572]	train-merror:0.392157	val-merror:0.399747
[573]	train-merror:0.3921	val-merror:0.399678
[574]	train-merror:0.392065	val-merror:0.399596
[575]	train-merror:0.392003	val-merror:0.399615
[576]	train-merror:0.391952	val-merror:0.399599
[577]	train-merror:0.391918	val-merror:0.399577
[578]	train-merror:0.39186	val-merror:0.399577
[579]	train-merror:0.391823	val-merror:0.399514
[580]	train-merror:0.391772	val-merror:0.399408
[581]	train-merror:0.3917	val-merror:0.399405
[582]	train-merror:0.391632	val-merror:0.399351
[583]	train-merror:0.391577	val-merror:0.399339
[584]	train-merror:0.391554	val-merror:0.399307
[585]	train-merror:0.391448	val-merror:0.399273
[586]	train-merror:0.391403	val-merror:0.399251
[587]	train-merror:0.391369	val-merror:0.399226
[588]	train-merror:0.391302	val-merror:0.39916

[740]	train-merror:0.384479	val-merror:0.394123
[741]	train-merror:0.384439	val-merror:0.394117
[742]	train-merror:0.384391	val-merror:0.394198
[743]	train-merror:0.384362	val-merror:0.39417
[744]	train-merror:0.384315	val-merror:0.394157
[745]	train-merror:0.384273	val-merror:0.394151
[746]	train-merror:0.384223	val-merror:0.39407
[747]	train-merror:0.384177	val-merror:0.394022
[748]	train-merror:0.384157	val-merror:0.393985
[749]	train-merror:0.384115	val-merror:0.393963
[750]	train-merror:0.384071	val-merror:0.393969
[751]	train-merror:0.384003	val-merror:0.393982
[752]	train-merror:0.383951	val-merror:0.393988
[753]	train-merror:0.383925	val-merror:0.393913
[754]	train-merror:0.383883	val-merror:0.393872
[755]	train-merror:0.383861	val-merror:0.393831
[756]	train-merror:0.38381	val-merror:0.393825
[757]	train-merror:0.383807	val-merror:0.393775
[758]	train-merror:0.38378	val-merror:0.393787
[759]	train-merror:0.383713	val-merror:0.393743
[760]	train-merror:0.383659	val-merror:0.393

[912]	train-merror:0.37854	val-merror:0.390575
[913]	train-merror:0.378473	val-merror:0.390594
[914]	train-merror:0.378448	val-merror:0.390676
[915]	train-merror:0.378417	val-merror:0.390604
[916]	train-merror:0.378383	val-merror:0.390591
[917]	train-merror:0.378343	val-merror:0.39055
[918]	train-merror:0.378314	val-merror:0.390538
[919]	train-merror:0.37829	val-merror:0.390516
[920]	train-merror:0.378252	val-merror:0.39055
[921]	train-merror:0.378191	val-merror:0.390532
[922]	train-merror:0.378133	val-merror:0.390453
[923]	train-merror:0.378099	val-merror:0.390409
[924]	train-merror:0.378078	val-merror:0.390356
[925]	train-merror:0.378064	val-merror:0.390406
[926]	train-merror:0.378057	val-merror:0.39034
[927]	train-merror:0.378016	val-merror:0.390346
[928]	train-merror:0.378001	val-merror:0.390274
[929]	train-merror:0.377972	val-merror:0.390284
[930]	train-merror:0.377958	val-merror:0.390321
[931]	train-merror:0.377908	val-merror:0.390321
[932]	train-merror:0.37785	val-merror:0.39025

[1082]	train-merror:0.373345	val-merror:0.387718
[1083]	train-merror:0.373301	val-merror:0.387749
[1084]	train-merror:0.373294	val-merror:0.387765
[1085]	train-merror:0.373266	val-merror:0.387771
[1086]	train-merror:0.373256	val-merror:0.38774
[1087]	train-merror:0.373229	val-merror:0.387702
[1088]	train-merror:0.37317	val-merror:0.387684
[1089]	train-merror:0.37316	val-merror:0.38768
[1090]	train-merror:0.373138	val-merror:0.387668
[1091]	train-merror:0.37309	val-merror:0.387593
[1092]	train-merror:0.373055	val-merror:0.387615
[1093]	train-merror:0.373024	val-merror:0.38763
[1094]	train-merror:0.373002	val-merror:0.38763
[1095]	train-merror:0.372982	val-merror:0.387646
[1096]	train-merror:0.372921	val-merror:0.38763
[1097]	train-merror:0.372925	val-merror:0.387652
[1098]	train-merror:0.372889	val-merror:0.38769
[1099]	train-merror:0.372877	val-merror:0.387702
[1100]	train-merror:0.372837	val-merror:0.387706
[1101]	train-merror:0.372835	val-merror:0.38768
[1102]	train-merror:0.372817	v

# 测试

In [ ]:
testData = sparse.load_npz('../trainTestData/csr_testData9521.npz')

In [ ]:
testData.shape

In [ ]:
# xgb.to_graphviz(bst, num_trees=1)

In [ ]:
pred_prob = np.zeros((testData.shape[0],7))
for k in tqdm(range(10), desc='k-flod loop'):
    param = {}

    param['gpu_id'] = 1
    # # param['n_gpus'] = -1
    param['max_bin'] = 512
    param['grow_policy'] = 'depthwise'
    param['tree_method'] = 'gpu_hist'


    param['eta'] = 0.1
    param['min_child_weight'] = 0.8
    param['subsample'] = 0.8

    param['lambda'] = 0.6
#     param['gamma'] = 0.1
    param['max_depth'] = 4
    param['silent'] = 1
    param['nthread'] = 48
    # param['top_k'] = 1500
    # param['scale_pos_weight'] = 9
    # param['missing'] = -999.0

    param['num_class'] = 7
    param['objective'] = 'multi:softprob'
    bst = xgb.Booster(param)
    bst.load_model('../model/save_k_flod_9521_'+str(k)+'.model')
    
    xg_test = xgb.DMatrix(testData)
    pred_prob +=  bst.predict(xg_test)

In [ ]:
pred_prob = pred_prob / 10

In [ ]:
testRowIndex = pd.read_csv("../trainTestData/testRowIndex.csv", header = None)
pred_prob_all = np.loadtxt('../processed/pred_prob_flod_621270.txt', dtype=float, delimiter=',')
testRowIndex = testRowIndex.loc[:, 0].values

In [ ]:
for i in range(testRowIndex.shape[0]):
    pred_prob_all[testRowIndex[i]] = pred_prob[i]
#     pred_prob_all[testRowIndex[i]] = (pred_prob[i] + pred_prob_all[testRowIndex[i]])/2

In [ ]:
np.savetxt('../processed/pred_prob_flod_9521.txt',pred_prob_all, fmt='%s', delimiter=',', newline='\n')

In [ ]:
pred_label = np.argmax(pred_prob_all, axis=1)
pred_label.shape

In [ ]:
age_test['label'] = pred_label
submission = age_test.rename(columns = {0: 'id'})

submission.to_csv('../data/xgb_k_9521_submission.csv', index=0)